In [9]:
from batchgenerators.utilities.file_and_folder_operations import *
from _warnings import warn
from tqdm import trange
from time import time, sleep
from collections import OrderedDict
from typing import Tuple
import torch.backends.cudnn as cudnn
import numpy as np
import torch
from nnunet.training.data_augmentation.data_augmentation_moreDA import get_moreDA_augmentation
from nnunet.training.loss_functions.deep_supervision import MultipleOutputLoss2
from nnunet.utilities.to_torch import maybe_to_torch, to_cuda
from nnunet.network_architecture.generic_UNet import Generic_UNet
from nnunet.network_architecture.initialization import InitWeights_He
from nnunet.network_architecture.neural_network import SegmentationNetwork
from nnunet.training.data_augmentation.default_data_augmentation import default_2D_augmentation_params, \
    get_patch_size, default_3D_augmentation_params
from nnunet.training.dataloading.dataset_loading import unpack_dataset
from nnunet.training.network_training.nnUNetTrainer import nnUNetTrainer
from nnunet.utilities.nd_softmax import softmax_helper
from sklearn.model_selection import KFold
from torch import nn
from torch.cuda.amp import autocast
from nnunet.training.learning_rate.poly_lr import poly_lr
from batchgenerators.utilities.file_and_folder_operations import *

/media/oem/2f92ffc8-5cb4-410e-a267-5f20ebf74ea1/miniconda3/envs/wxg/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

nnUNet_raw_data_base is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
RESULTS_FOLDER is not defined and nnU-Net cannot be used for training or inference. If this is not in

In [13]:
network = Generic_UNet(3, 32, 2,
                                    6,
                                    2, 2, nn.Conv2d, nn.InstanceNorm2d, {'eps': 1e-5, 'affine': True}, nn.Dropout2d,
                                    {'p': 0, 'inplace': True},
                                    nn.LeakyReLU, {'negative_slope': 1e-2, 'inplace': True}, True, False, lambda x: x, InitWeights_He(1e-2),
                                   [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]], [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3]], False, True, True)


In [14]:
x = torch.randn(2,3,256,256)

In [16]:
a=network(x)

In [26]:
a[5].shape

torch.Size([2, 2, 8, 8])